In [1]:
!pip list | grep dbt   

In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import platform
import logging

sys.path.append('..')
from common import db_operations

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,execute_query

In [2]:
# Configure basic logging for the business logic file
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.11.13


In [36]:
conn = connect_to_trino()
query = f"""
select * FROM adhoc.facts_prod.braking_report
"""
df = execute_query(conn, query)
df.head()

2025-09-17 06:17:25 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-09-17 06:17:25 - INFO - ✅ STEP 1: Connected to Trino
2025-09-17 06:17:25 - INFO - ⚙️ Executing query...
2025-09-17 06:18:07 - INFO - ✅ Query executed successfully!


,vehicle_id,start,end,max_bpp,avg_bpp,ttl_dist_m,start_vel,peak_vel,avg_decel_mps2,braking_force_kgf
0,27,2025-09-16 18:19:37,2025-09-16 18:19:46,45.20,36.35,34.66,38.60,38.60,1.18,1629.08
1,27,2025-09-16 04:04:18,2025-09-16 04:04:25,40.40,28.64,17.86,34.52,34.52,1.35,1853.73
2,27,2025-09-16 06:37:30,2025-09-16 06:37:39,29.20,21.07,33.23,31.72,31.72,0.90,1232.90
3,27,2025-09-16 07:08:26,2025-09-16 07:08:34,36.80,27.10,25.09,44.30,44.30,1.55,2128.18
4,27,2025-09-16 03:29:54,2025-09-16 03:30:04,33.20,26.73,25.42,35.87,35.87,1.04,1437.67


In [39]:
df[df.vehicle_id.isin(['3'])].sort_values(by='start')

,vehicle_id,start,end,max_bpp,avg_bpp,ttl_dist_m,start_vel,peak_vel,avg_decel_mps2,braking_force_kgf
3940,3,2025-07-25 00:02:01,2025-07-25 00:02:11,45.60,29.44,33.21,30.30,30.30,0.91,1256.87
3942,3,2025-07-25 00:03:42,2025-07-25 00:03:51,55.20,40.68,41.37,40.61,40.61,1.22,1682.40
3935,3,2025-07-25 00:08:10,2025-07-25 00:08:20,55.60,35.72,36.76,38.48,38.48,1.10,1520.27
3968,3,2025-07-25 00:15:07,2025-07-25 00:15:16,45.60,25.60,39.58,31.52,31.52,0.95,1307.54
3936,3,2025-07-25 00:18:00,2025-07-25 00:18:09,60.40,39.04,28.43,34.01,34.01,0.99,1360.45
...,...,...,...,...,...,...,...,...,...,...
4519,3,2025-08-27 23:08:35,2025-08-27 23:08:45,31.20,22.80,36.52,31.88,31.88,0.89,1226.39
4578,3,2025-08-27 23:14:45,2025-08-27 23:14:54,32.40,24.27,41.44,36.18,36.18,1.06,1459.53
4559,3,2025-08-27 23:19:04,2025-08-27 23:19:12,65.20,34.36,21.81,30.48,30.48,0.96,1321.49
4581,3,2025-08-27 23:33:59,2025-08-27 23:34:09,36.00,23.80,24.08,33.89,33.89,1.00,1381.78


In [ ]:
conn = connect_to_trino()
query = f"""
select * FROM adhoc.facts_prod.braking_report
"""
df = execute_query(conn, query)
df.head()

In [40]:
conn = connect_to_trino()
query = f"""
select * FROM adhoc.facts_prod.energy_mileage_report
"""
df = execute_query(conn, query)
df.head()

2025-09-17 06:27:30 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-09-17 06:27:30 - INFO - ✅ STEP 1: Connected to Trino
2025-09-17 06:27:30 - INFO - ⚙️ Executing query...
2025-09-17 06:27:33 - INFO - ✅ Query executed successfully!


,vehicle_id,date,dist_travelled_km,energy_consumed_kwh,regen_energy_kwh,idling_energy_kwh,net_energy_kwh,mileage_kwh_per_km
0,11,2025-09-15,413.88,406.65,92.97,0.38,313.68,0.76
1,7,2025-09-14,830.00,865.90,251.18,3.85,614.72,0.74
2,7,2025-09-16,459.38,432.79,125.77,4.65,307.02,0.67
3,27,2025-09-14,0.00,0.81,0.00,0.23,0.81,inf
4,13,2025-09-15,377.25,392.04,87.09,1.02,304.95,0.81


In [42]:
df.size

6704

In [41]:
df[df.vehicle_id.isin(['3'])].sort_values(by='date')

,vehicle_id,date,dist_travelled_km,energy_consumed_kwh,regen_energy_kwh,idling_energy_kwh,net_energy_kwh,mileage_kwh_per_km
396,3,2025-07-25,37.62,45.44,8.93,1.72,36.51,0.97
813,3,2025-07-25,37.62,45.44,8.93,1.72,36.51,0.97
797,3,2025-07-26,805.62,620.01,90.74,1.36,529.27,0.66
377,3,2025-07-26,805.62,620.01,90.74,1.36,529.27,0.66
792,3,2025-07-27,423.38,355.43,62.85,4.10,292.58,0.69
...,...,...,...,...,...,...,...,...
423,3,2025-09-14,656.00,625.28,108.55,19.92,516.73,0.79
411,3,2025-09-15,656.12,637.99,98.74,7.48,539.25,0.82
823,3,2025-09-15,656.12,637.99,98.74,7.48,539.25,0.82
399,3,2025-09-16,0.50,1.71,0.08,65.05,1.63,3.26


In [18]:
df.dtypes

vehicle_id                  object
start_time          datetime64[ns]
end_time            datetime64[ns]
charge_dur                   int64
soc_start                  float64
soc_end                    float64
tpc_kwh                    float64
soc_kwh                    float64
diff_kwh_percent           float64
dtype: object

In [19]:
df[df.diff_kwh_percent>10]

,vehicle_id,start_time,end_time,charge_dur,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kwh_percent
6,11,2025-09-15 02:40:14.026,2025-09-15 03:08:15.526,1679,59.2,80.0,104.26,87.98,16.93
16,15,2025-09-16 20:44:07.962,2025-09-16 21:06:39.462,839,94.0,100.0,22.85,25.38,10.50
21,14,2025-09-15 02:52:39.455,2025-09-15 03:21:11.035,1709,60.4,80.4,98.26,84.60,14.94
23,16,2025-09-16 18:53:40.849,2025-09-16 21:00:06.195,7583,16.4,100.0,297.89,353.63,17.11
27,12,2025-09-16 14:49:16.889,2025-09-16 16:30:46.396,6088,68.0,96.0,143.94,118.44,19.44
...,...,...,...,...,...,...,...,...,...
2500,14,2025-07-25 20:26:06.973,2025-07-25 20:38:48.315,760,94.4,98.4,23.10,16.92,30.88
2503,11,2025-07-24 09:42:14.036,2025-07-24 09:44:48.916,153,41.2,41.2,0.39,0.00,200.00
2504,9,2025-07-24 14:58:53.908,2025-07-24 14:58:58.121,3,67.6,67.6,0.00,0.00,200.00
2506,6,2025-09-15 11:43:59.973,2025-09-15 12:10:07.133,1448,63.6,80.0,76.73,69.37,10.07


In [17]:
df[df.vehicle_id.isin(['3','16','18','19'])]

,vehicle_id,start_time,end_time,charge_dur,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kwh_percent
0,3,2025-09-15 17:23:16.250,2025-09-15 18:53:52.065,5432,36.0,100.0,285.13,270.72,5.18
12,16,2025-09-16 06:37:44.449,2025-09-16 08:24:17.071,6389,34.0,100.0,293.46,279.18,4.99
23,16,2025-09-16 18:53:40.849,2025-09-16 21:00:06.195,7583,16.4,100.0,297.89,353.63,17.11
33,16,2025-09-15 15:03:54.768,2025-09-15 15:39:31.528,2135,53.2,63.6,44.98,43.99,2.22
36,16,2025-09-15 18:30:41.290,2025-09-15 20:11:01.530,6018,30.4,100.0,284.38,294.41,3.47
...,...,...,...,...,...,...,...,...,...
2486,3,2025-07-26 07:04:31.795,2025-07-26 08:10:24.192,3951,36.0,62.0,114.17,109.98,3.73
2489,16,2025-07-26 17:02:07.142,2025-07-26 17:53:37.362,3089,77.6,100.0,104.43,94.75,9.72
2491,16,2025-07-26 16:16:53.062,2025-07-26 16:44:59.182,1685,64.8,77.6,56.82,54.14,4.82
2492,3,2025-09-15 05:49:43.327,2025-09-15 07:13:43.807,5038,37.6,100.0,275.66,263.95,4.34


In [ ]:
start_date = '2023-01-01'
end_date = '2023-02-15'
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
print(filtered_df)

In [23]:
start_date = '2025-09-01'
end_date = '2025-09-15'

In [29]:
start_date = '2025-09-01'
end_date = '2025-09-15' # Although end_date is defined, it's not used in the provided snippet.

# Ensure 'start_time' is in datetime format if it isn't already
df['start_time'] = pd.to_datetime(df['start_time'])

# Extract the date part from 'start_time'
df.loc[:, 'date'] = df['start_time'].dt.date

# Apply the filters correctly
filtered_df = df[
    (df['vehicle_id'].isin(['3', '16', '18', '19'])) &
    (df['start_time'] > start_date)
]

# Perform the groupby and size aggregation
result = filtered_df.groupby(['vehicle_id', 'date']).size().reset_index(name='count_of_instances')

In [35]:
filtered_df[(filtered_df.diff_kwh_percent>10) & (filtered_df.soc_start !=	filtered_df.soc_end)].sort_values(by='start_time')

,vehicle_id,start_time,end_time,charge_dur,soc_start,soc_end,tpc_kwh,soc_kwh,diff_kwh_percent,date
408,16,2025-09-01 18:03:05.074,2025-09-01 18:05:27.333,141,22.4,22.8,1.19,1.69,34.74,2025-09-01
1421,16,2025-09-01 18:03:05.074,2025-09-01 18:05:27.333,141,22.4,22.8,1.19,1.69,34.74,2025-09-01
349,16,2025-09-05 06:08:00.312,2025-09-05 06:13:07.212,303,37.6,37.2,0.13,1.69,170.67,2025-09-05
1372,16,2025-09-05 06:08:00.312,2025-09-05 06:13:07.212,303,37.6,37.2,0.13,1.69,170.67,2025-09-05
283,16,2025-09-06 05:39:58.451,2025-09-06 06:07:42.691,1545,90.8,100.0,45.12,38.92,14.76,2025-09-06
1342,16,2025-09-06 05:39:58.451,2025-09-06 06:07:42.691,1545,90.8,100.0,45.12,38.92,14.76,2025-09-06
234,19,2025-09-09 17:25:11.588,2025-09-09 17:28:52.048,152,52.8,53.2,2.46,1.69,37.14,2025-09-09
1166,19,2025-09-09 17:25:11.588,2025-09-09 17:28:52.048,152,52.8,53.2,2.46,1.69,37.14,2025-09-09
50,19,2025-09-14 06:11:44.553,2025-09-14 06:16:18.713,274,61.6,63.6,9.68,8.46,13.41,2025-09-14
1571,19,2025-09-14 06:11:44.553,2025-09-14 06:16:18.713,274,61.6,63.6,9.68,8.46,13.41,2025-09-14
